In [1]:
import numpy as np

from pymatgen import Molecule
from pymatgen.io.babel import BabelMolAdaptor

from polymers.core import Polymer
from polymers.utils import get_topology

Set the start, bulk and terminal molecules. Also set the forcefield species name mapping

In [2]:
# start molecule
peo_start = Molecule.from_file("PEOmonomer_start.xyz")
ff_map = ["Ce", "H", "H", "H", "O", "Cm", "H", "H"]
peo_start.add_site_property("ff_map", ff_map)
s_head = 0
s_tail = 5

# chain molecule
peo_bulk = Molecule.from_file("PEOmonomer_bulk.xyz")
ff_map = ["Ce", "H", "H", "O", "Cm", "H", "H"]
peo_bulk.add_site_property("ff_map", ff_map)
head = 0
tail = 4

# terminal molecule
peo_end = Molecule.from_file("PEOmonomer_end.xyz")
ff_map = ["Ce", "H", "H", "O", "Cm", "H", "H", "H"]
peo_end.add_site_property("ff_map", ff_map)
e_head = 0
e_tail = 4

Linear polymer chain

In [3]:
n_units = 8
link_distance = 1.0

# create the polymer
peo_polymer_linear = Polymer(peo_start, s_head, s_tail, 
                      peo_bulk, head, tail, 
                      peo_end, e_head, e_tail, 
                      n_units, link_distance, linear_chain=True)

peo_polymer_linear.molecule.to(filename="polymer_linear.xyz", fmt="xyz")

length:  1
length:  2
length:  3
length:  4
length:  5
length:  6
length:  7
length:  8
length:  9


Random walk polymer chain

In [4]:
n_units = 10
link_distance = 1.0

# create the polymer
peo_polymer = Polymer(peo_start, s_head, s_tail, 
                      peo_bulk, head, tail, 
                      peo_end, e_head, e_tail, 
                      n_units, link_distance)

peo_polymer.molecule.to(filename="polymer.xyz", fmt="xyz")

move 1
length:  1
move 5
length:  2
move 1
length:  3
move 6
length:  4
move 5
move 1
length:  5
move 1
length:  6
move 5
length:  7
move 4
length:  8
move 6
length:  9
move 4
move 4
move 6
length:  10
move 2
move 2
move 3
move 1
length:  11


In [5]:
atoms, bonds, angles, dihedrals = get_topology(peo_polymer_linear.molecule)
print set([x[2] for x in bonds])
print set([x[3] for x in angles])
print set([x[4] for x in dihedrals])

set([('Ce', 'O'), ('Ce', 'H'), ('Ce', 'Cm'), ('Cm', 'H'), ('Cm', 'O')])
set([('Cm', 'H', 'Ce'), ('Ce', 'Cm', 'H'), ('Cm', 'O', 'Ce'), ('Ce', 'Cm', 'O'), ('Ce', 'H', 'H'), ('Cm', 'O', 'H'), ('O', 'Ce', 'Cm'), ('Ce', 'H', 'O'), ('Cm', 'H', 'H')])
set([('Ce', 'Cm', 'O', 'O'), ('Ce', 'Cm', 'H', 'H'), ('Ce', 'Cm', 'H', 'O'), ('Ce', 'Cm', 'Cm', 'O'), ('Ce', 'Ce', 'Cm', 'O')])


In [6]:
atoms, bonds, angles, dihedrals = get_topology(peo_polymer.molecule)
print set([x[2] for x in bonds])
print set([x[3] for x in angles])
print set([x[4] for x in dihedrals])

set([('Cm', 'H'), ('Ce', 'O'), ('Ce', 'Cm'), ('O', 'O'), ('Ce', 'H'), ('Cm', 'O')])
set([('O', 'Ce', 'O'), ('Cm', 'H', 'Ce'), ('Ce', 'Cm', 'H'), ('Cm', 'O', 'Ce'), ('Ce', 'O', 'Cm'), ('Ce', 'Cm', 'O'), ('Ce', 'O', 'H'), ('Ce', 'H', 'H'), ('Cm', 'O', 'H'), ('O', 'Ce', 'Cm'), ('Ce', 'H', 'O'), ('Cm', 'H', 'H')])
set([('Ce', 'H', 'O', 'O'), ('Ce', 'Cm', 'Cm', 'O'), ('Ce', 'Cm', 'H', 'H'), ('Ce', 'Cm', 'H', 'O'), ('Ce', 'Ce', 'Cm', 'O'), ('Ce', 'Cm', 'O', 'O')])


Use openbabel to optimize the polymer chain.
Local optimization is used to preserve the chain structure.

In [7]:
peo_obabel = BabelMolAdaptor(peo_polymer.molecule)
# ['gaff', 'ghemical', 'mmff94', 'mmff94s', 'uff']
peo_obabel.localopt(forcefield='mmff94', steps=500)
peo_optimized = peo_obabel.pymatgen_mol

Add the site property "ff_map" to the openbabel optimized polymer chain

In [8]:
ff_map = [site.ff_map for site in peo_polymer.molecule]
peo_optimized.add_site_property("ff_map", ff_map)
peo_optimized.to(filename="polymer_optimized.xyz", fmt="xyz")

Get the polymer topology: atoms, bonds, angles and dihedrals

In [9]:
atoms, bonds, angles, dihedrals = get_topology(peo_optimized)

print len(atoms), len(bonds), len(angles), len(dihedrals)

86 70 107 88


Use Packmol to pack the optimized polymer chains into a box

In [10]:
from rubicon.io.packmol.packmol import PackmolRunner 

packmol_config = [{"number": 1, "fixed": [0, 0, 0, 0, 0, 0],"centerofmass": ""},
                  {"number": 30, "inside sphere": [0, 0, 0, 200.0]}]
pmr = PackmolRunner([peo_optimized, peo_optimized],
                    packmol_config,
                    tolerance=2.0,
                    filetype="xyz",
                    control_params={"nloop": 1000},
                    auto_box=False, output_file="poly_packed.xyz")
packed_polymer = pmr.run()

packed molecule written to poly_packed.xyz
